In [1]:
import tensorflow as tf
# Fixes bad convolution
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [41]:
s = tf.io.read_file('day13.txt')

@tf.function
def solve1(s):
    s = tf.strings.split(s, '\n')
    t0 = tf.strings.to_number(s[0])
    busnumbers = tf.strings.split(s[1], ',')
    busnumbers = tf.strings.to_number(busnumbers[busnumbers != 'x'])

    # Removing 1 from busnumbers so we can catch the buses leaving at the same time as t0
    quotients = (t0-1)//busnumbers
    earliest_times = (quotients+1)*busnumbers
    idx = tf.argmin(earliest_times)
    bus_id = busnumbers[idx]
    t1 = earliest_times[idx]
    delta = t1 - t0

    return bus_id*delta

%timeit solve1(s)
solve1(s)

1.4 ms ± 83.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


<tf.Tensor: shape=(), dtype=float32, numpy=102.0>

In [103]:
s = tf.io.read_file('day13.txt')

def mul_inv(a, b):
    b0 = b
    x0, x1 = tf.constant(0, tf.int64), tf.constant(1, tf.int64)
    if b == 1:
        return tf.constant(1, tf.int64)
    while a > 1:
        q = a // b
        a, b = b, a%b
        x0, x1 = x1 - q * x0, x0
    if x1 < 0: x1 += b0
    return x1

@tf.function
def solve2(s):
    s = tf.strings.split(s, '\n')
    busnumbers = tf.strings.split(s[1], ',')

    offsets = -tf.range(tf.shape(busnumbers)[0], dtype=tf.int64)[busnumbers != 'x']
    busnumbers = tf.strings.to_number(busnumbers[busnumbers != 'x'], out_type=tf.int64)
    N = tf.reduce_prod(busnumbers)
    inverses = tf.vectorized_map(lambda bn: mul_inv(N//bn, bn), busnumbers)
    
    return tf.reduce_sum(offsets*inverses*(N//busnumbers)) % N

%timeit solve2(s)
solve2(s)

6.16 ms ± 441 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


<tf.Tensor: shape=(), dtype=int64, numpy=327300950120029>